## LH_구내식당_식수인원_예측

## Environment

In [ ]:
!pip install catboost
!pip install kaggler
!pip install pendulum
!pip install flaml
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import datetime
np.random.seed(0)

from kaggler.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

from tqdm.notebook import tqdm
import os, re
import glob
import calendar

from flaml import AutoML
import statsmodels.api as sm
import pendulum

## Load Data

In [ ]:
path = '/content/drive/MyDrive/구내식당/water/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
holiday = pd.read_csv(path+'holidays_old.csv')

df = pd.concat([train.iloc[:, :-2], test])
target_df = train.iloc[:, -2:]
df.columns = ['일자', '요일', '정원','휴가자', '출장자', '야근자',\
                 '재택근무자', '조식', '중식', '석식']

## Data Preprocessing

### Menu Feature Preprocessing

In [ ]:
# Menu-extracting function
def extractMenu(array, keywords=[], not_in_keywords={}, comm_not_in=[]):
  extractedMenu = []
  for menu_nm in array:
    for kw in keywords:
      if menu_nm.find(kw) > -1:
        has_not_in = False
        if kw in not_in_keywords:
          for sub_kw in not_in_keywords[kw]:
            if menu_nm.find(sub_kw) > -1:
              has_not_in = True
              break
        for sub_kw in comm_not_in:
            if menu_nm.find(sub_kw) > -1:
              has_not_in = True
              break

        if not has_not_in:
          extractedMenu.append(menu_nm)
          break
  return(extractedMenu)

def extractMenu2(array, keywords=[]):
  extractedMenu = []
  for menu_nm in tot_menu_arr:
    for kw in keywords:
      if menu_nm.find(kw) > -1:
        menu_nm_list = re.split(r'[^\w]', menu_nm)
        for menu_nm_tmp in menu_nm_list:
          if menu_nm_tmp.find(kw) + len(kw) == len(menu_nm_tmp): # 끝에 있으면
            extractedMenu.append(menu_nm)
        break
  return(extractedMenu)

In [ ]:
lunch_menu_data = df['중식']
dinner_menu_data = df['석식']

In [ ]:
tot_menu_arr = []
pattern = r"\(.*\)"
for menu_data in [lunch_menu_data, dinner_menu_data]:
  for daily_menu in menu_data:
    menu_list = daily_menu.strip().split()
    menu_list2 = []
    for i, menu_nm in enumerate(menu_list):
      menu_nm = re.sub(pattern, '', menu_nm)
      if menu_nm.strip() in ['', '*']:
        continue
      if menu_nm[0] == '(' or menu_nm[-1] == ')':
        continue
      menu_list2.append(menu_nm)
    tot_menu_arr += menu_list2

In [ ]:
tot_menu_arr = set(tot_menu_arr)

In [ ]:
len(tot_menu_arr)

In [ ]:
# 육류 분류
# 소고기
# https://namu.wiki/w/%EC%87%A0%EA%B3%A0%EA%B8%B0
beef = ['소고기', '쇠고기', '불고기', '떡갈비', '갈비찜', '소갈비', '육사시미', '육회', '장조림', '와규', '야키니쿠', '규동', '스테이크', '햄버그 스테이크',
 '함박스테이크', '함바그스테이크', '함박 스테이크', '햄버거', '로스트 디너', '비프가스밀라네사', '웰링턴', '슈하스쿠', '아사도', '우육면',
 '육개장', '육포', '평양냉면', '비프 스트로가노프', '설렁탕', '소고기국', '소머리국밥', '곰탕', '너비아니', '보르챠', '소꼬리']
# 돼지고기
# https://namu.wiki/w/%EB%8F%BC%EC%A7%80%EA%B3%A0%EA%B8%B0
pig = ['돼지', '돼지머리', '머릿고기', '뒷고기', '관자살', '콧등살', '삼각살', '설중살', '설하살', '안중살', '뽈항정살',
 '볼살', '두항정', '돼지코', '항정살', '목살', '가브리살', '갈비', '앞다리살', '갈매기살', '등심', '안심',
 '삼겹살', '오겹살', '뒷다리살', '돈족', '내장', '오소리감투', '허파', '염통', '콩팥', '새끼보', '돈낭',
 '돈족', '돼지꼬리', '사태', '막창', '감자탕', '돈가스', '돼지갈비', '돼지국밥', '돼지불고기', '두루치기', '순대',
 '순댓', '족발', '보쌈', '수육', '편육', '제육', '탕수육', '삼겹', '맥적', '차슈', '향우구육', '꿔바로우', '훙사오러우',
 '회과육', '동파육', '라후테', '오향장육', '슈바인스학세', '소시지', '소세지', '포크 커틀릿', '함바그 스테이크', '함바그스테이크',
 '함박스테이크', '살스테이크','살 스테이크', '함박 스테이크', '베이컨', '햄', '스팸', '폭립', '폭찹', '돈지루', '부타동', '바쿠테', '팟 카파오 무 쌉', '비엔나', '소떡', '육']
# 닭고기
# https://namu.wiki/w/%EB%8B%AD%EA%B3%A0%EA%B8%B0
chicken = ['닭', '깐풍기', '꼬꼬면', '궁보계정', '간장닭', '기스면', '계', '도빙무시', '라조기', '백숙', '영계백숙',
 '불닭', '삼계탕', '삼계선', '오니시메', '옻닭', '연팔기', '유린기', '육회', '좌종당계', '찜닭', '초계밀면',
 '치킨', '도리텐', '지파이', '치짜', '취계', '카라아게', '가라아', '파닭', '양파닭', '케밥', '코코뱅', '탕수기',
 '포계', '프랑구 아사두']
# 양고기
# https://namu.wiki/w/%EC%96%91%EA%B3%A0%EA%B8%B0
sheep = ['양고기','훠궈', '양꼬치', '케밥', '샤슬릭', '징기스칸', '셰퍼드 파이', '허르헉', '양갈비']
# 오리고기
# https://namu.wiki/w/%EC%98%A4%EB%A6%AC%EA%B3%A0%EA%B8%B0
dug = ['오리']

web_keywords = beef + pig + chicken + sheep + dug
keywords = ['돈까스', '히레카츠', '히레까쓰', '히레가스', '포크', '부대찌개', '뒷다리', '앞다리', '돈', '순살',
                '소머리', '등뼈', '곱창', '도가니', '뼈해장국', '뼈다귀해장국', '목심', '채끝', '우둔', '양지', '설도', '만두', '만둣',
                '잡채', '류산슬', '유산슬', '고기', '고깃']
keywords += web_keywords

not_in_keywords = {'오리':['아오리', '오리엔탈'], '계':['계란', '계발', '계피'], '장조림':['계란', '메추리알'], '치킨':['치킨무'], '돈':['돈나물'], '만두':['당면계란'], '만둣':['당면계란']}
meat_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 돼지고기
keywords = ['돼지', '돼지머리', '머릿고기', '뒷고기', '관자살', '콧등살', '삼각살', '설중살', '설하살', '안중살', '뽈항정살',
 '볼살', '두항정', '돼지코', '항정살', '목살', '가브리살', '앞다리살', '갈매기살', '등심', '안심',
 '삼겹살', '오겹살', '앞다리살', '뒷다리살', '돈족', '내장', '오소리감투', '허파', '염통', '콩팥', '새끼보', '돈낭',
 '돈족', '돼지꼬리', '사태', '막창', '감자탕', '돈가스', '돼지갈비', '돼지국밥', '돼지불고기', '두루치기', '순대',
 '순댓', '족발', '보쌈', '수육', '편육', '제육', '탕수육', '삼겹', '맥적', '차슈', '향우구육', '꿔바로우', '훙사오러우',
 '회과육', '동파육', '라후테', '오향장육', '슈바인스학세', '소시지', '소세지', '포크 커틀릿',
 '목살스테이크','목살 스테이크', '베이컨', '햄', '스팸', '폭립', '폭찹', '돈지루', '부타동', '바쿠테', '팟 카파오 무 쌉', '비엔나', '소떡',
 '돈까스', '히레카츠', '히레까쓰', '히레가스', '포크', '돈', '등뼈', '뼈해장국', '뼈다귀해장국']
not_in_keywords = {'돈':['돈나물'], '만두':['당면계란'], '만둣':['당면계란']}
pig_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 소고기
keywords = ['소고기', '쇠고기', '소불고기', '소갈비', '육사시미', '육회', '와규', '야키니쿠', '규동', '소곱창',
            '로스트 디너', '비프가스밀라네사', '웰링턴', '슈하스쿠', '아사도', '우육면',
            '육개장', '육포', '평양냉면', '비프 스트로가노프', '설렁탕', '소고기국', '소머리국밥', '곰탕', '너비아니', '보르챠', '소꼬리', '소머리', '설도', '목심', '채끝', '우둔', '양지', '도가니']
beef_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 닭고기
keywords = ['닭', '깐풍기', '꼬꼬면', '궁보계정', '간장닭', '기스면', '계', '도빙무시', '라조기', '백숙', '영계백숙',
          '불닭', '삼계탕', '삼계선', '오니시메', '옻닭', '연팔기', '유린기', '육회', '좌종당계', '찜닭', '초계밀면',
          '치킨', '도리텐', '지파이', '치짜', '취계', '카라아게', '가라아', '파닭', '양파닭', '케밥', '코코뱅', '탕수기',
          '포계', '프랑구 아사두']
not_in_keywords = {'계':['계란', '계발', '계피'], '치킨':['치킨무']}

chicken_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 양고기 - 데이터 없어서 제외
keywords = ['양고기','훠궈', '양꼬치', '케밥', '샤슬릭', '징기스칸', '셰퍼드 파이', '허르헉', '양갈비']
sheep_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])
sheep_menus

In [ ]:
# 오리고기
keywords = ['오리']
not_in_keywords = {'오리':['아오리', '오리엔탈']}
duck_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
#난류 (계란)
keywords = ['계란', '난', '란', '메추리알', '날치알', '동태알']
not_in_keywords = {"란":['토란'], '난':['커리', '카레']}
egg_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 죽류
keywords = ['죽', '누룽지']
juk_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 덮밥 및 국밥류
keywords = ['덮밥', '국밥']
gukbob_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 비빔밥 및 볶음밥류
keywords = ['비빔밥', '볶음밥']
bb_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 국탕류
keywords = ['국', '탕', '찌개', '국물']
soup_menus = extractMenu2(tot_menu_arr, keywords=keywords)

In [ ]:
# 구이류
keywords = ['구이']
gui_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 전류
keywords = ['전', '부침개', '빈대떡']
jeon_menus = extractMenu2(tot_menu_arr, keywords=keywords)

In [ ]:
# http://yaksik.net/detail.php?number=24904
# 튀김류
keywords = ['튀김', '까스', '카츠', '가츠', '까츠', '탕수', '덴뿌라', '덴푸라', '크로켓', '고로케', '맛탕', '치킨', '통닭', '부각', '강정', '김말이', '깐풍']
fry_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

#### 메뉴 추가 특성 - Part1

In [ ]:
# 곡물
keywords = ['현미', '밥', '쌀', '보리', '죽', '참깨', '들깨', '수수', '잡곡', '귀리', '퀴노아', '아마란스', '옥수수', '기장', '메밀', '모밀']
grain_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 콩류
keywords = ['콩', '녹두', '팥', '완두']
not_in_keywords = {'콩':['콩나물']}
bean_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 묵
keywords = ['묵']
not_in_keywords = {'묵':['어묵', '묵은지']}
kor_jelly_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 생선 및 조개류
# https://ko.wikipedia.org/wiki/%EC%83%9D%EC%84%A0
# https://namu.wiki/w/%EC%83%9D%EC%84%A0
# https://namu.wiki/w/%EC%A1%B0%EA%B0%9C
keywords = ['생선', '조개', '메기', '송어', '오징어', '굴', '멸치', '숭어', '성게', '고등어', '명태',
            '쏨뱅이', '연어', '틸라피아', '우럭', '이리치', '가재', '참바리', '상어', '돔',
            '삼치', '방어', '참치', '새우', '문어', '홍어', '농어', '붉평치', '청상아리', '황새치',
            '다랑어', '비막치어', '장어', '녹새치', '숭어', '굴비', '조기', '갈치', '꽁치',
            '전어', '명태', '노가리', '황태', '은어', '가물치', '쏘가리', '붕어', '잉어', '모래마주', '가자미',
            '간재미', '가오리', '박대', '양미리', '과메기', '청어', '생태',
            '개복치', '광어', '넙치', '기름치', '까나리', '날치','놀래미'
            ,'능성어','달고기','대구','도다리','도루묵','도미','독가시치'
            ,'만새기','망상어','문절망둑','물메기','미꾸라지','민어','방어'
            ,'추어탕','배스','밴댕이','뱅어','벵에돔','병어','보리멸'
            ,'복어','볼락','부세','부시리','붕장어','블루길'
            ,'빙어','산천어','서대','시샤모','쏘가리','쏠배감펭','쏨뱅이'
            ,'아귀','아구','임연수','전갱이','전복치','점성어','정어리'
            ,'준치','쥐치','청새치','청어','향어','홍어','황새치','매운탕'
            ,'루테피스크','게맛살','물회','회덮밥','부야베스','북엇국','세꼬시','수르스트뢰밍','식해','어묵','오뎅'
            ,'쥐포','추어탕','피시 앤드 칩스','피쉬 앤드 칩스','피시앤드칩스','피쉬앤드칩스','피시앤칩스','피쉬앤칩스','해물'
            ,'가리비', '개오지', '꼬막','대칭이','바지락','백합','홍합','소라', '골뱅이', '고둥','재첩'
            ,'전복','플라티케라무스', '봉골레', '클램차우더']
not_in_keywords = {'굴':['굴소스'], '새우':['새우젓']}
fish_shell_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 채소류
# https://namu.wiki/w/%EC%B1%84%EC%86%8C?from=%EC%95%BC%EC%B1%84
keywords = ['가지', '갓', '감자', '고구마', '고사리', '고추', '페페론치노', '냉이', '근대', '깻잎', '차조기'
            , '당근', '더덕', '도라지', '동아', '딸기', '마', '마늘', '멜론', '무', '무청'
            , '바나나', '배추', '버섯', '부추', '브로콜리', '상추', '생강', '쇠비름', '나물'
            , '쑥', '시금치', '수박', '시호', '아스파라거스', '야콘', '양파', '여주', '연근', '열무', '오이'
            , '우엉', '인삼', '죽순', '청경채', '참외', '칡', '풋콩', '토란', '토마토', '쪽파', '대파', '파인애플'
            , '파프리카', '피망', '케일', '고수', '로즈마리', '루타바가', '바질', '박하', '산마늘', '셀러리'
            , '아티초크', '타임', '파슬리', '호박', '피클', '파채', '파김치', '채소', '야채']
not_in_keywords = {'무':['무침'], '마':'마카로니', '고추':['고추장']}
vegetable_menus =  extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 해조류
# https://namu.wiki/w/%EC%A1%B0%EB%A5%98(%EC%88%98%EC%A4%91%EC%83%9D%EB%AC%BC)?from=%ED%95%B4%EC%A1%B0%EB%A5%98
keywords = ['김', '우뭇가사리', '한천', '매생이', '파래', '바다포도', '해캄', '클로렐라', '청각', '마리모모스볼', '다시마', '미역', '감태', '톳']
not_in_keywords = {'김':['김치', '튀김', '김칫']}
sea_alg_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 발효된 콩 상품 -> 장류
# https://namu.wiki/w/%EC%9E%A5%EB%A5%98
keywords = ['된장', '간장', '쯔유', '노추', '미소', '고추장', '청국장', '담북장', '팥장', '두부장', '비지장', '어육장', '춘장', '마장', '낫토', '두반장', '해선장', '굴소스', '게장',
 '장조림', '양념장', '장국', '쌈장', '초장', '*장']
jang_menus  = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 김치
# https://namu.wiki/w/%EA%B9%80%EC%B9%98
keywords = ['김치', '깍두기', '석박지', '동치미', '겉절이', '묵은지', '소박이', '섞박지', '생채', '게국지', '김칫']
kimchi_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 만두
# https://namu.wiki/w/%EB%A7%8C%EB%91%90
keywords = ['만두', '춘권', '만쥬', '사모사', '만둣']
mandu_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords={}, comm_not_in=[])

In [ ]:
# 곡물 가루(밀가루, 쌀가루 등 전분)
# https://namu.wiki/w/%EB%B0%80%EA%B0%80%EB%A3%A8
# 미숫가루
keywords = ["면", "수제비", "전", "부침개", "빵", "춘권", "튀김", "과자", "국수", "메밀", "모밀", "피자", "전병", "떡", "어묵", "오뎅", "소시지", "소세지", "햄", "김밥", 
            "부대찌개", "스콘", "만두", "파이", "빈대떡", "케이크", "케익", "쿠키", "핫도그", "파스타", "치킨", "라자냐", "팟타이", "나쵸", "팝콘", '스파게티', '짬뽕']
not_in_keywords = {'치킨':['치킨무'], '전':['전주식'], '짬뽕':['고기', '찌개', '국']}
powder_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=[])

In [ ]:
# 과일
# https://namu.wiki/w/%EA%B3%BC%EC%9D%BC
# https://namu.wiki/w/%EC%88%98%EC%9E%85%20%EA%B3%BC%EC%9D%BC
keywords = ['구기자','매실','무화과','버찌','체리','복분자','복숭아','블랙베리','블루베리','딸기','살구','앵두','자두','포도'
            ,'감','다래','대추','머루','모과','무화과','배','사과','석류','으름','귤','유자','레드향','천혜향','한라봉'
            ,'과라나','구아바','구즈베리','토마토','나랑히야','노니','노팔','니파팜','두꾸','두리안','라임','람부탄'
            ,'레몬','애플','루비솔트부쉬','리치','여지','마랑','마룰라','마르멜로','마프랑','망고','블랙베리','아보카도'
            ,'아로니아','아사이베리','아사이 베리','양초열매','오렌지','올리브','용안','롱간','자몽','바나나','딸기','수박'
            ,'참외','멜론','메론','여주','파인애플','토마토','코코넛','크랜베리','타마린드','파파야','패션프루트','패션후르츠']
not_in_keywords = {'살구':['구이', '목살', '삼겹살', '가브리살', '갈비살', '항정살'], '감':['감자'], '배':['배추', '알배기', '소배기']}
comm_not_in = ['주스', '쥬스', '음료', 'D', '순']
fruit_menus = extractMenu(tot_menu_arr, keywords=keywords, not_in_keywords=not_in_keywords, comm_not_in=comm_not_in)

#### 메뉴 추가 특성 - Part2

In [ ]:
# 쌀
# https://namu.wiki/w/%EA%B3%A1%EB%AC%BC

keywords = ['쌀', '잡곡', '오곡', '현미', '흑미', '귀리', '차조', '렌틸콩', '강낭콩', '병아리콩', '완두콩', '기장', '보리', '수수', '호밀'] 
not_in_keywords = {'쌀':['쌀국수', '찹쌀'], '기장':['장조림'], '수수':['옥수수', '부꾸미']} # 찹쌀은 밥 메뉴명에 쓰이지 않아 삭제
comm_not_in = ['스프']
rice_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 김밥 및 초밥

keywords = ['김밥', '초밥'] 
not_in_keywords = {'김밥':['볶음밥']}
comm_not_in = []
gimbab_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 소금, 식초 등에 절인 해산물
# 해산물이 들어있지 않은 절임류도 포함시킴

keywords = ['절임', '젓'] 
not_in_keywords = {}
comm_not_in = []
saused_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 면류
# https://femiwiki.com/w/%EB%B6%84%EB%A5%98:%EC%A2%85%EB%A5%98/%EB%A9%B4%EC%9A%94%EB%A6%AC
keywords = ['국수', '면', '파스타', '스파게티', '짬뽕', '라면'] 
not_in_keywords = {'짬뽕':['고기', '찌개', '국']}
comm_not_in = []
noodle_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 스튜 - 조림과 찌개의 중간단계
# https://namu.wiki/w/%EC%8A%A4%ED%8A%9C
keywords = ['스튜', '조림'] 
not_in_keywords = {}
comm_not_in = []
stew_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 한국 전통 샐러드
keywords = ['나물', '무침'] 
not_in_keywords = {'나물':['콩나물', '밥']}
comm_not_in = []
namul_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 피클
keywords = ['피클'] 
not_in_keywords = {}
comm_not_in = []
pickle_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 뚝배기 - 없음 -> 제외
keywords = ['뚝배기', '돌솥'] 
not_in_keywords = {}
comm_not_in = []
dduk_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

dduk_menus

In [ ]:
# 샐러드
keywords = ['샐러드'] 
not_in_keywords = {}
comm_not_in = []
salad_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 우유
# 우유가 들어간 식재료(크림, 요거트 등)종류로 변경
keywords = ['까르보나라', '크림', '요거트'] 
not_in_keywords = {}
comm_not_in = ['샐러드', 'D', '드레싱', '소스'] # 샐러드 드레싱, 디핑소스 제외
milk_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 빵, 쿠키
# https://ko.wikipedia.org/wiki/%EB%B9%B5_%EB%AA%A9%EB%A1%9D
keywords = ['와플', '케이크', '케잌', '바게트', '도넛', '도너츠', '핫도그', '도라야키', '베이글', '번', '비스킷', '스콘', '토스트', '브레드', '포카차', '피자', '호두과자', '쿠키'] 
not_in_keywords = {}
comm_not_in = []
bread_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
# 음료
# https://ko.wikipedia.org/wiki/%EB%B9%B5_%EB%AA%A9%EB%A1%9D
keywords = ['주스', '쥬스', '수정과', '식혜', '식초', '코코아', '칵테일', '스무디', '우유', '셰이크', '야쿠르트', '요구르트', '커피', '차', '탄산수', '음료'] 
not_in_keywords = {'차':['차돌']}
comm_not_in = []
drink_menus = extractMenu(tot_menu_arr, keywords, not_in_keywords, comm_not_in)

In [ ]:
def get_food_one_hot(x, menu_array):
  menu_list = x.strip().split()
  for i, menu_nm in enumerate(menu_list):
    menu_nm = re.sub(pattern, '', menu_nm)
    if menu_nm.strip() in ['', '*']:
      continue
    if menu_nm[0] == '(' or menu_nm[-1] == ')':
      continue
    try:
      if menu_array.index(menu_nm) > -1:
        return 1
    except Exception:
      pass
  return 0

In [ ]:
# 데이터 병합
menu_col_nm = ['육류', '난류', '죽류', '덮밥_국밥류', '비빔밥_볶음밥류', '국탕류', '구이류', '전류', '튀김류', '곡물', '콩류',
               '묵', '생선_조개류', '채소류', '해조류', '장류', '김치', '만두', '곡물가루', '과일', '쌀', '김밥_초밥', '절임류',
               '면류', '스튜', '나물_무침류', '피클', '샐러드', '우유', '빵류', '음료', '돼지고기', '소고기', '닭고기', '오리고기']
menu_data_arr = [meat_menus, egg_menus, juk_menus, gukbob_menus, bb_menus, soup_menus, gui_menus, jeon_menus, fry_menus, grain_menus, bean_menus,
                 kor_jelly_menus, fish_shell_menus, vegetable_menus, sea_alg_menus, jang_menus, kimchi_menus, mandu_menus, powder_menus, fruit_menus, rice_menus, gimbab_menus, saused_menus,
                 noodle_menus, stew_menus, namul_menus, pickle_menus, salad_menus, milk_menus, bread_menus, drink_menus, pig_menus, beef_menus, chicken_menus, duck_menus]

for col_type in ['중식메뉴', '석식메뉴']:
  for i, menu_arr in enumerate(menu_data_arr):
    train[col_type + '_' + menu_col_nm[i]] = train[col_type].apply(lambda x: get_food_one_hot(x, menu_arr))
    test[col_type + '_' + menu_col_nm[i]] = train[col_type].apply(lambda x: get_food_one_hot(x, menu_arr))

In [ ]:
df = pd.concat([train, test], axis=0).reset_index(drop=True)
df = df.fillna(0)
df.columns = ['일자', '요일', '정원', '휴가자', '출장자', '야근자', '재택근무자', '조식', '중식', '석식',\
              '중식계', '석식계', '중식메뉴_육류',
       '중식메뉴_난류', '중식메뉴_죽류', '중식메뉴_덮밥_국밥류', '중식메뉴_비빔밥_볶음밥류', '중식메뉴_국탕류',
       '중식메뉴_구이류', '중식메뉴_전류', '중식메뉴_튀김류', '중식메뉴_곡물', '중식메뉴_콩류', '중식메뉴_묵',
       '중식메뉴_생선_조개류', '중식메뉴_채소류', '중식메뉴_해조류', '중식메뉴_장류', '중식메뉴_김치', '중식메뉴_만두',
       '중식메뉴_곡물가루', '중식메뉴_과일', '중식메뉴_쌀', '중식메뉴_김밥_초밥', '중식메뉴_절임류', '중식메뉴_면류',
       '중식메뉴_스튜', '중식메뉴_나물_무침류', '중식메뉴_피클', '중식메뉴_샐러드', '중식메뉴_우유', '중식메뉴_빵류',
       '중식메뉴_음료', '중식메뉴_돼지고기', '중식메뉴_소고기', '중식메뉴_닭고기', '중식메뉴_오리고기', '석식메뉴_육류',
       '석식메뉴_난류', '석식메뉴_죽류', '석식메뉴_덮밥_국밥류', '석식메뉴_비빔밥_볶음밥류', '석식메뉴_국탕류',
       '석식메뉴_구이류', '석식메뉴_전류', '석식메뉴_튀김류', '석식메뉴_곡물', '석식메뉴_콩류', '석식메뉴_묵',
       '석식메뉴_생선_조개류', '석식메뉴_채소류', '석식메뉴_해조류', '석식메뉴_장류', '석식메뉴_김치', '석식메뉴_만두',
       '석식메뉴_곡물가루', '석식메뉴_과일', '석식메뉴_쌀', '석식메뉴_김밥_초밥', '석식메뉴_절임류', '석식메뉴_면류',
       '석식메뉴_스튜', '석식메뉴_나물_무침류', '석식메뉴_피클', '석식메뉴_샐러드', '석식메뉴_우유', '석식메뉴_빵류',
       '석식메뉴_음료', '석식메뉴_돼지고기', '석식메뉴_소고기', '석식메뉴_닭고기', '석식메뉴_오리고기']
df.drop(columns=['조식', '중식', '석식'], inplace=True)
# 중간 변수 색출
df = df.drop(columns=['중식메뉴_죽류', '중식메뉴_곡물', '중식메뉴_김치', '중식메뉴_쌀', '중식메뉴_나물_무침류', '중식메뉴_피클'
                        , '석식메뉴_죽류', '석식메뉴_곡물', '석식메뉴_김치', '석식메뉴_쌀', '석식메뉴_나물_무침류', '석식메뉴_피클',\
                      '중식메뉴_음료', '석식메뉴_음료'])

### 날씨 데이터 추가

In [ ]:
weather_path = '/content/drive/MyDrive/구내식당/water/날씨'

w_attrs = ['3시간기온', '강수형태', '강수확률', '습도', '6시간강수량']
w_years = os.listdir(weather_path)

In [ ]:
def get_wdata(data_path) :
    datetime_list = []
    value_list_12 = []
    value_list_18 = []
    curr_mon = ''

    with open(data_path, 'r') as f :
        lines = f.readlines()
        for i, line in enumerate(lines) :
            if line.strip() == '' :
                break
            row_data = line.strip().split(',')
            row_data = [elem.strip() for elem in row_data]
            if i == 0 :
                curr_mon = row_data[-1].split()[-1][:-2]
                continue
            if len(row_data) == 1 :
                curr_mon =row_data[-1].split()[-1][:-2]
                continue
            r_day, r_hour, r_fore, r_temp = row_data
            if '6시간' not in data_path :
                if r_hour == '1400' and r_fore in ['+13', '+19']:
                    if r_fore in '+13' :
                        datetime_list.append(curr_mon[:4]+'-'+curr_mon[4:]+'-'+str('%02d'%int(r_day)))
                    if r_fore == '+13' :
                        value_list_12.append(float(r_temp))
                    else :
                        value_list_18.append(float(r_temp))
            else :
                if r_hour == '1400' and r_fore in ['+13', '+19']:
                    if r_fore in '+13' :
                        datetime_list.append(curr_mon[:4]+'-'+curr_mon[4:]+'-'+str('%02d'%int(r_day)))
                    if r_fore == '+13' :
                        value_list_12.append(float(r_temp))
                    else :
                        value_list_18.append(float(r_temp))

        if data_path == '/content/drive/MyDrive/구내식당/water/정훈/날씨/2017/충무공동_강수형태_201701_201712.csv' :
            nan_val = [0]*22
            nan_val[-7] = 1
            nan_val[-6] = 1
            nan_val[-2] = 1 
            value_list_12.extend(nan_val)
            value_list_18.extend(nan_val)
    return datetime_list, value_list_12, value_list_18

In [ ]:
# ['3시간기온', '강수형태', '강수확률', '습도', '하늘상태']
w_data_rain_12 = []
w_data_rain_18 = []
w_data_rtype_12 = []
w_data_rtype_18 = []
w_data_hum_12 = []
w_data_hum_18 = []
w_data_temp_12 = []
w_data_temp_18 = []
w_data_rain_total_12 = []
w_data_rain_total_18 = []
w_datetime = []

for year in w_years :
    w_subdir = os.path.join(weather_path, year)
    file_names = os.listdir(w_subdir)
    file_name = ""
    if year != '2021' :
        file_name = f'{year}01_{year}12.csv'
    else :
        file_name = f'{year}01_{year}06.csv'
    file_path_rain = os.path.join(w_subdir, '충무공동_강수확률_'+file_name)
    file_path_temp = os.path.join(w_subdir, '충무공동_3시간기온_'+file_name)
    file_path_hum = os.path.join(w_subdir, '충무공동_습도_'+file_name)
    file_path_rtype = os.path.join(w_subdir, '충무공동_강수형태_'+file_name)
    file_path_rain_total = os.path.join(w_subdir, '충무공동_6시간강수량_'+file_name)

    datetime_list_temp, value_list_temp_12, value_list_temp_18 = get_wdata(file_path_temp) # 3시간기온 데이터
    datetime_list_rain, value_list_rain_12, value_list_rain_18 = get_wdata(file_path_rain) # 강수확률 데이터
    datetime_list_hum, value_list_hum_12, value_list_hum_18 = get_wdata(file_path_hum) # 습도 데이터
    datetime_list_rtype, value_list_rtype_12, value_list_rtype_18 = get_wdata(file_path_rtype) # 강수형태 데이터
    datetime_list_rain_total, value_list_rain_total_12, value_list_rain_total_18 = get_wdata(file_path_rain_total) # 강수량 데이터

    w_datetime   += datetime_list_temp
    w_data_rain_12  += value_list_rain_12
    w_data_temp_12  += value_list_temp_12
    w_data_hum_12   += value_list_hum_12
    w_data_rtype_12 += value_list_rtype_12
    w_data_rain_18  += value_list_rain_18
    w_data_temp_18  += value_list_temp_18
    w_data_hum_18   += value_list_hum_18
    w_data_rtype_18 += value_list_rtype_18
    w_data_rain_total_12 += value_list_rain_total_12
    w_data_rain_total_18 += value_list_rain_total_18

In [ ]:
w_df = pd.DataFrame({'일자':pd.Series(w_datetime, dtype='datetime64[ns]'),
                   'rain_prob_lunch':pd.Series(w_data_rain_12, dtype='float'),
                   'temp_lunch':pd.Series(w_data_temp_12, dtype='float'),
                   'hum_lunch':pd.Series(w_data_hum_12, dtype='float'),
                   'rain_total_lunch':pd.Series(w_data_rain_total_12, dtype='float'),
                   'rain_type_lunch':pd.Series(w_data_rtype_12, dtype='float'),
                   'rain_prob_dinner':pd.Series(w_data_rain_18, dtype='float'),
                   'temp_dinner':pd.Series(w_data_temp_18, dtype='float'),
                   'hum_dinner':pd.Series(w_data_hum_18, dtype='float'),
                   'rain_total_dinner':pd.Series(w_data_rain_total_18, dtype='float'),
                   'rain_type_dinner':pd.Series(w_data_rtype_18, dtype='float'),}).sort_values('일자').reset_index(drop=True)

In [ ]:
# 불쾌지수 컬럼 추가
# https://dacon.io/competitions/official/235736/codeshare/2753?page=1&dtype=recent
w_df['discomfort_lunch'] = 1.8*w_df['temp_lunch'] - 0.55*(1-w_df['hum_lunch']/100)*(1.8*w_df['temp_lunch']-26) + 32
w_df['discomfort_dinner'] = 1.8*w_df['temp_dinner'] - 0.55*(1-w_df['hum_dinner']/100)*(1.8*w_df['temp_dinner']-26) + 32

# 불쾌지수 명목변수화 
w_df['discomfort_lunch'] = w_df['discomfort_lunch'].apply(lambda x : 0 if x<68 else (1 if 68<=x<75 else (2 if 75<=x<80 else 3)))
w_df['discomfort_dinner'] = w_df['discomfort_dinner'].apply(lambda x : 0 if x<68 else (1 if 68<=x<75 else (2 if 75<=x<80 else 3)))

# 진주에는 눈이 거의 내리지 않음 -> 비로 고쳐주겠음
w_df['rain_type_lunch'] = w_df['rain_type_lunch'].apply(lambda x : 0 if x==0 else 1)
w_df['rain_type_dinner'] = w_df['rain_type_dinner'].apply(lambda x : 0 if x==0 else 1)

w_df['rain_prob_lunch'] = w_df['rain_prob_lunch'].apply(lambda x : np.round(x/100, 1))
w_df['rain_prob_dinner'] = w_df['rain_prob_dinner'].apply(lambda x : np.round(x/100, 1))
w_df.drop(columns=['temp_lunch', 'temp_dinner', 'hum_lunch', 'hum_dinner'], inplace=True)

In [ ]:
w_df['일자'] = w_df['일자'].shift(-1)
df['일자'] = df['일자'].astype('datetime64')
df = pd.merge(df, w_df, on='일자', how='left')

### 파생 변수 - Feature_pipeline

In [ ]:
def feature_pipeline(df) :
    df['식사가능인원'] = df['정원']-(df['휴가자']+df['출장자']+df['재택근무자'])+df['야근자']
    df['년'] = df['일자'].dt.year
    df['월'] = df['일자'].dt.month
    df['일'] = df['일자'].dt.day
    df['년월'] = df['년'].astype('str')+'_'+df['월'].astype('str')

    first_dayofmonth = []
    last_dayofmonth = []
    for i in df['년월'].unique() :
        first_dayofmonth.append(df[df['년월']==i].iloc[0].name)
        last_dayofmonth.append(df[df['년월']==i].iloc[-1].name)
    df['첫_출근일'] = df.apply(lambda x : 1 if x.name in first_dayofmonth else 0, axis=1)
    df['마지막_출근일'] = df.apply(lambda x : 1 if x.name in last_dayofmonth else 0, axis=1)

    # 월_주차 컬럼 추가
    df['주차'] = df['일자'].apply(lambda x: pendulum.parse(str(x)).week_of_month)
    repair_2017 = df[(df['년']==2017)&(df['주차']<0)]['일자'].dt.week
    repair_2021 = df[(df['년']==2021)&(df['주차']<0)]['일자'].dt.week
    df['주차'][list(repair_2017.index)] = repair_2017.values
    df['주차'][list(repair_2021.index)] = repair_2021.values
    df['주차'][[709, 954, 955]] = np.array([6, 5, 5])

    month_to_season = {1: 3,2: 3,3:0,4:0,5:0,6:1,7:1,8:1,9:2,10:2,11:2,12: 3}
    df['계절'] = df['월'].apply(lambda x : month_to_season[x])
    df['요일'] = df['일자'].dt.weekday
    df['야근_가능'] = df['요일'].apply(lambda x : 1 if (x==2) or (x==4) else 0)
    df['is_corona'] = df['일자'].apply(lambda x : 0 if x < pd.to_datetime('2020-01-06') else 1)
    df['연기준몇주째']= df['일자'].dt.weekofyear
    df['월마지막일여부'] =df['일자'].dt.is_month_end
    df['월일수']= df['일자'].dt.days_in_month

    # 공휴일 데이터 추가
    holiday['date'] = pd.to_datetime(holiday['date'])
    df['before_holiday'] = df['일자'].apply(lambda x : 1 if (x+dt.timedelta(1) in holiday['date'].tolist()) else 0)
    df['after_holiday'] = df['일자'].apply(lambda x : 1 if (x-dt.timedelta(1) in holiday['date'].tolist()) else 0)
    
    # 탄력근무제 적용 여부
    test_date="20180701"
    convert_date = datetime.datetime.strptime(test_date, "%Y%m%d").date()
    df['탄력근무_여부'] = df['일자'].apply(lambda x : 1 if x >= convert_date else 0)

    # # 이벤트 데이터 고민 - 복날 / 연말
    bok = pd.to_datetime(['2016-08-16', '2016-07-27', '2016-07-18', '2017-08-11', '2017-07-21','2017-07-12', '2018-08-16', '2018-07-27', '2018-07-17', '2019-08-12', '2019-07-22', '2019-07-12', '2020-07-27', '2020-07-16'])
    df['복날'] = df['일자'].apply(lambda x : 1 if x in bok else 0)

    # end_year = df[(df['월']==12)&(df['일']>=21)].index
    # df['연말'] = df.apply(lambda x : 1 if  x.name in end_year else 0, axis=1)

    # 명절 전 영업일 여부
    event = pd.to_datetime(['2016-02-05', '2019-09-13', '2017-01-26','2017-09-29', '2018-02-14', '2018-09-21', '2019-02-01', '2019-09-11', '2020-01-23', '2020-09-30', '2021-02-10'])
    df['명절_이전_영업일'] = df['일자'].apply(lambda x : 1 if x in event else 0)

    # 명절 전이나 둘 중에 하나만 써야함 / 특히 10월달이 이상함
    # df['긴_연휴'] = df['일자'].diff().dt.days.fillna(0).astype('int')
    # df['연휴_뒤'] = df['긴_연휴'].apply(lambda x: 1 if x>=4 else 0)
    # df['연휴_앞'] = df.apply(lambda x : 1 if x.name in df[df['연휴_뒤']==1].index-1 else 0, axis=1)
    # df.drop(columns=['긴_연휴'], inplace=True)

    # 국정 감사 기간
    gukgam = pd.to_datetime(['2016-10-04', '2016-10-05', '2016-10-06', '2017-10-10', '2017-10-11', '2017-10-12', '2017-10-13', '2018-10-08', '2018-10-10', '2018-10-11',\
                             '2019-10-02', '2019-10-04'])
    df['국정감사'] = df['일자'].apply(lambda x : 1 if x in gukgam else 0)

    # 인원 변화
    # df['인원변화'] = df['정원'].diff()
    # df['인원변화'][0] = 0

    df.drop(columns=['정원', '년월'], inplace=True)
    return df

In [ ]:
df = feature_pipeline(df)

### 이상치 제거
* 수요일 야근자가 100명 이상, 정책 변경 후 금요일 100명 이상 제거
* 식사비율이 이상치에 해당하는 값 제거
    * 1.5*(제 3분위 수 - 제 1분위 수)

In [ ]:
df['중식비율'] = df['중식계']/df['식사가능인원']
df['석식비율'] = df['석식계']/df['식사가능인원']

In [ ]:
df = df[~df.index.isin(df[(df['요일']==2)&(df['야근자']>=100)].index.append(df[(df['요일']==4)&(df['일자']>='2019-01-01')&(df['야근자']>100)].index))]

train = df.iloc[:-50]
train['월마지막일여부'] = train['월마지막일여부'].astype(int)
test = df.iloc[-50:]

train_2 = train[train['석식계']!=0]
iqr_lunch = 1.5*(train['중식비율'].quantile(0.75) - train['중식비율'].quantile(0.25))
iqr_dinner = 1.5*(train_2['석식비율'].quantile(0.75) - train_2['석식비율'].quantile(0.25))

train_1 = train[(train['중식비율']>=train['중식비율'].quantile(0.25)-iqr_lunch)|(train['중식비율']<=train['중식비율'].quantile(0.75)+iqr_lunch)]
train_2 = train_2[train_2['일자']!='2016-10-05']

## Encoding

In [ ]:
def get_one_hot(x, target_val):
  if x == target_val:
    return 1
  else:
    return 0

In [ ]:
from sklearn.preprocessing import LabelEncoder
onehot_col = ['년', '월', '요일', '계절']
# onehot_col = ['월', '요일', '계절']
# df_tmp = df.copy()
# df = pd.concat([df[list((set(df.columns)-set(onehot_col)))],\
#                 pd.get_dummies(df[onehot_col])], axis=1)

sub_types = [[2016, 2017, 2018, 2019, 2020, 2021], [1,2,3,4,5,6,7,8,9,10,11,12], [0,1,2,3,4], [0,1,2,3]]
# sub_types = [[1,2,3,4,5,6,7,8,9,10,11,12], [0,1,2,3,4], [0,1,2,3]]

train_lun = train_1.copy()
train_din = train_2.copy()
# for i, col_type in enumerate(onehot_col):
#   for j, class_nm in enumerate(sub_types[i]):
#     df_tmp[col_type + '_' + str(class_nm)] = df_tmp[col_type].apply(lambda x: get_one_hot(x, class_nm))

for i, col_type in enumerate(onehot_col):
  for j, class_nm in enumerate(sub_types[i]):
    train_lun[col_type + '_' + str(class_nm)] = train_lun[col_type].apply(lambda x: get_one_hot(x, class_nm))
for i, col_type in enumerate(onehot_col):
  for j, class_nm in enumerate(sub_types[i]):
    train_din[col_type + '_' + str(class_nm)] = train_din[col_type].apply(lambda x: get_one_hot(x, class_nm))

In [ ]:
train_1 = train_lun[[i for i in train_lun.columns if ('석식' not in i) and ('dinner' not in i) and ('탄력근무' not in i) and ('명절' not in i)]]
train_2 = train_din[[i for i in train_din.columns if ('중식' not in i) and ('lunch' not in i)]]

In [ ]:
for i, col_type in enumerate(onehot_col):
  for j, class_nm in enumerate(sub_types[i]):
    test[col_type + '_' + str(class_nm)] = test[col_type].apply(lambda x: get_one_hot(x, class_nm))

In [ ]:
train_1 = train_1.drop(columns=['일자', '요일', '월', '월마지막일여부', '계절', '월일수','복날', '중식비율', '년', '국정감사'])
train_2 = train_2.drop(columns=['일자', '요일', '월', '월마지막일여부', '월일수','계절', '복날', 'after_holiday', '석식비율', '년', '첫_출근일', '국정감사'])

In [ ]:
train_1.columns

In [ ]:
train_2.columns

## Flaml Model

In [ ]:
import numpy as np
import shap
def get_columns(model, X_test, train, del_col_nm='석식계'):
  explainer = shap.Explainer(model._model)
  shap_values = explainer(X_test)
  vals = np.abs(shap_values.values).mean(0)
  cols = list(train.columns)
  cols.remove(del_col_nm)
  feature_importance = pd.DataFrame(list(zip(cols, vals)), columns=['col_name','feature_importance_vals'])
  feature_importance.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
  return feature_importance[feature_importance['feature_importance_vals'] > 1]['col_name'].values

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

def run_kfold(train, test, selected_cols=[], target_col='중식계'):
    folds=KFold(n_splits=5, shuffle=True, random_state=2021)
    outcomes=[]
    model = None
    sub=[]

    # X = None
    # X_test = None
    # if len(selected_cols) > 0:
    #   X = np.array(train[selected_cols])
    #   X_test = np.array(test[selected_cols])
    # else:
    #   X = np.array(train.drop(columns=[target_col]))
    #   cols = list(train.columns)
    #   cols.remove(target_col)
    #   X_test = np.array(test[cols])

    # y = np.array(train[target_col])
    X = np.array(train.drop(columns=[target_col]))
    cols = list(train.columns)
    cols.remove(target_col)
    X_test = np.array(test[cols])
    y = np.array(train[target_col])

    # 정규화
    standardScaler = StandardScaler()
    standardScaler.fit(X)
    X = standardScaler.transform(X)

    X_test = standardScaler.transform(X_test) # 정규화
    cols_import = []

    for n_fold, (train_index, val_index) in enumerate(folds.split(X)):
        print(n_fold, 'fold started =========================================')
        # X_train1, X_val = , X_train.loc[val_index]
        y_train = y[train_index]
        X_train = X[train_index]
        # y_train1, y_val = y_train.loc[train_index], y_train.loc[val_index]
        y_val = y[val_index]
        X_val = X[val_index]
        if n_fold == 0:
          automl = AutoML()
          automl_settings = {
              "time_budget": 120,  # in seconds
              "metric": 'mae',
              "task": 'regression'
          }
          automl.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **automl_settings)
          model = automl.model
        else:
          model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                        eval_metric='mae', early_stopping_rounds=10) 
        
        pred1 = model.predict(X_test)
        if len(selected_cols) == 0:
          cols_import += list(get_columns(model, X_test, train=train, del_col_nm=target_col))
        sub.append(pred1)
    return sub, model, X_test, set(cols_import)

In [ ]:
my_submission, model, X_test, cols_import = run_kfold(train_1, test, selected_cols=[], target_col='중식계')

In [ ]:
pred1 = np.mean(my_submission, axis=0)

In [ ]:
my_submission, model, X_test, cols_import = run_kfold(train_2, test, selected_cols=[], target_col='석식계')

In [ ]:
pred2 = np.mean(my_submission, axis=0)

## Submission

In [ ]:
sample_submission = pd.read_csv(path+'sample_submission.csv')
submission = sample_submission.copy()

In [ ]:
submission['중식계'] = pred1
submission['석식계'] = pred2

In [ ]:
from datetime import datetime
now_tm = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')

sub_path = '/content/drive/MyDrive/DACON/Dacon_Industry_Meal/submit/'
submission.to_csv(sub_path+now_tm+'.csv', index=False)

## DACON API

In [ ]:
!pip install /content/drive/MyDrive/구내식당/water/dacon_submit_api-0.0.4-py3-none-any.whl

In [ ]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/content/drive/MyDrive/DACON/Dacon_Industry_Meal/submit/2021-07-22-02:37:54.csv', 
'c7eb6960dc46f188d012c09ac2fbde1a0f90af05eeef92249719473834bb981e', 
'235743', 
'datu', 
'optuna')